In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
import argparse
from joblib import dump
import matplotlib.pyplot as plt
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import eli5
from eli5.sklearn import PermutationImportance
from sklearn import tree

In [2]:
base_dir = '../data/Car Hacking Dataset/exp_factor_0.095/'

In [3]:
file_name = 'smart_output.csv'
data = pd.read_csv(os.path.join(base_dir, file_name))

data = data.assign(IAT=data['Timestamp'].diff().fillna(0))
data.drop(['Timestamp'], axis = 1, inplace = True)

In [4]:
data.head()

,ID,DLC,Payload,label,IAT
0,848.0,8.0,3.716925e+17,0.0,0.000000
1,704.0,8.0,1.441152e+18,0.0,0.000221
2,1072.0,8.0,0.000000e+00,0.0,0.000554
3,1201.0,8.0,0.000000e+00,0.0,0.000238
4,497.0,8.0,0.000000e+00,0.0,0.000248


In [5]:
def hex_to_bin(hex_num):
    
    binary_value = bin(int(str(hex_num), 16))[2:]
    
    return binary_value

def int_to_bin(int_num):
    
    binary_value = bin(int_num)[2:]
    
    return binary_value

def pad(value, length):
    
    curr_length = len(str(value))
    
    zeros = '0' * (length - curr_length)
    
    return zeros + value

hex_to_dec = lambda x: int(x, 16)

def transform_data(data):

    data['ID'] = data['ID'].apply(hex_to_dec)
    data['Payload'] = data['Payload'].apply(hex_to_dec)

    return data

def sequencify_data(X, y, seq_size=10):
    max_index = len(X) - seq_size + 1

    X_seq = []
    y_seq = []

    for i in range(0, max_index, seq_size):
        X_seq.append(X[i:i+seq_size])  # Append the sequence from DataFrame 'X'
        y_seq.append(1 if 1 in y[i:i+seq_size] else 0)  # Check for '1' in 'y' values

    return np.array(X_seq), np.array(y_seq)

# data = transform_data(data)

In [6]:
X = data.drop('label', axis = 1)
y = data['label']

X_seq, y_seq = sequencify_data(X.values, y.values)

#Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=True)
X_seq_train, X_seq_test, y_seq_train, y_seq_test = train_test_split(X_seq, y_seq, test_size = 0.2, shuffle= True)

#Standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

seq_scaler = StandardScaler()
num_train_samples, seq_length, num_features = X_seq_train.shape
num_test_samples, _, _ = X_seq_test.shape

X_train_seq_reshaped = X_seq_train.reshape(num_train_samples, -1)
X_test_seq_reshaped = X_seq_test.reshape(num_test_samples, -1)

X_train_seq_scaled = seq_scaler.fit_transform(X_train_seq_reshaped)
X_test_seq_scaled = seq_scaler.transform(X_test_seq_reshaped)

# Reshape the scaled data back to the original shape
X_seq_train = X_train_seq_scaled.reshape(num_train_samples, seq_length, num_features)
X_seq_test = X_test_seq_scaled.reshape(num_test_samples, seq_length, num_features)

oversample = SMOTE()
X_train_smote, y_train_smote = oversample.fit_resample(X_train, y_train) 

In [7]:
indexes = np.where(y_test == 1)[0]
X_test_attack = X_test[indexes]
y_test_attack = y_test.iloc[indexes]

In [8]:
def build_mlp():
    mlp = Sequential()
    mlp.add(Input(shape = (4)))
    mlp.add(Dense(128, activation = 'relu'))
    mlp.add(Dense(128, activation = 'relu'))
    mlp.add(Dense(1, activation = 'sigmoid'))

    mlp.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
    
    return mlp


es = EarlyStopping(monitor = 'val_loss', patience = 15, restore_best_weights = True)

mlp = KerasClassifier(build_fn = build_mlp, epochs = 100, batch_size = 128, validation_split = 0.2, callbacks = [es])
mlp.fit(X_train_smote, y_train_smote)

2024-02-22 14:00:11.423336: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-02-22 14:00:11.454676: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-02-22 14:00:12.658955: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:86:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.74GiB deviceMemoryBandwidth: 836.37GiB/s
2024-02-22 14:00:12.659043: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2024-02-22 14:00:16.877393: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2024-02-22 14:00:16.877591: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2

2024-02-22 14:00:27.514774: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-02-22 14:00:27.573117: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2400000000 Hz


Epoch 1/100


2024-02-22 14:00:28.834778: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


9890/9890 [==============================] - 28s 2ms/step - loss: 0.0218 - accuracy: 0.9936 - val_loss: 5.5411e-04 - val_accuracy: 1.0000
Epoch 2/100
9890/9890 [==============================] - 19s 2ms/step - loss: 0.0017 - accuracy: 0.9997 - val_loss: 4.4130e-04 - val_accuracy: 1.0000
Epoch 3/100
9890/9890 [==============================] - 19s 2ms/step - loss: 0.0014 - accuracy: 0.9998 - val_loss: 7.7349e-04 - val_accuracy: 1.0000
Epoch 4/100
9890/9890 [==============================] - 19s 2ms/step - loss: 0.0012 - accuracy: 0.9998 - val_loss: 7.2816e-05 - val_accuracy: 1.0000
Epoch 5/100
9890/9890 [==============================] - 19s 2ms/step - loss: 0.0011 - accuracy: 0.9998 - val_loss: 1.9883e-04 - val_accuracy: 1.0000
Epoch 6/100
9890/9890 [==============================] - 19s 2ms/step - loss: 0.0012 - accuracy: 0.9998 - val_loss: 1.0916e-04 - val_accuracy: 1.0000
Epoch 7/100
9890/9890 [==============================] - 20s 2ms/step - loss: 9.6352e-04 - accuracy: 0.9998 - va

In [76]:
perm_mlp = PermutationImportance(mlp, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm_mlp, feature_names = X.columns.tolist())

1552/1552 [==============================] - 1s 947us/step - loss: 0.4736 - accuracy: 0.9954


Weight,Feature
0.0078 ± 0.0002,ID
0.0043 ± 0.0002,IAT
0.0022 ± 0.0002,DLC
0.0021 ± 0.0001,Payload


In [77]:
xgb = XGBClassifier()
xgb.fit(X_train_smote, y_train_smote)

perm_xgb = PermutationImportance(xgb, random_state = 1).fit(X_test_attack, y_test_attack)
eli5.show_weights(perm_xgb, feature_names = X.columns.tolist())

/home/cse/visitor/anwesh.visitor/.conda/envs/tf/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:19:22] WARNING: /croot/xgboost-split_1675119646044/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Weight,Feature
0.1514 ± 0.0160,ID
0.1450 ± 0.0091,IAT
0.0066 ± 0.0012,Payload
0 ± 0.0000,DLC


In [75]:
eli5.show_weights(xgb, feature_names = X.columns.tolist())

Weight,Feature
0.9330,IAT
0.0608,DLC
0.0060,ID
0.0002,Payload


In [8]:
dt = DecisionTreeClassifier(max_depth = 2)
dt.fit(X_train_smote, y_train_smote)

perm_dt = PermutationImportance(dt, random_state = 1).fit(X_test, y_test)
eli5.show_weights(perm_dt, feature_names = X.columns.tolist())

|--- feature_3 <= 0.62
|   |--- class: 0.0
|--- feature_3 >  0.62
|   |--- feature_3 <= 0.92
|   |   |--- class: 1.0
|   |--- feature_3 >  0.92
|   |   |--- class: 0.0



In [21]:
indexes[:10]

array([ 372,  628, 1049, 1561, 1640, 1645, 1743, 1893, 2096, 2144])

In [33]:
eli5.explain_prediction(dt, X_test[2])

Explanation(estimator='DecisionTreeClassifier(max_depth=3)', description='\nFeatures with largest coefficients.\n\nFeature weights are calculated by following decision paths in trees\nof an ensemble (or a single tree for DecisionTreeClassifier).\nEach node of the tree has an output score, and contribution of a feature\non the decision path is how much the score changes from parent to child.\nWeights of all features sum to the output score or proba of the estimator.\n\nCaveats:\n1. Feature weights just show if the feature contributed positively or\n   negatively to the final score, and does not show how increasing or\n   decreasing the feature value will change the prediction.\n2. In some cases, feature weight can be close to zero for an important feature.\n   For example, in a single tree that computes XOR function, the feature at the\n   top of the tree will have zero weight because expected scores for both\n   branches are equal, so decision at the top feature does not change the\n   expected score. For an ensemble predicting XOR functions it might not be\n   a problem, but it is not reliable if most trees happen to choose the same\n   feature at the top.\n', error=None, method='decision path', is_regression=False, targets=[TargetExplanation(target=0.0, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='x3', weight=0.5, std=None, value=-0.20692607503903263), FeatureWeight(feature='<BIAS>', weight=0.5, std=None, value=1.0)], neg=[], pos_remaining=0, neg_remaining=0), proba=1.0, score=None, weighted_spans=None, heatmap=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)

In [17]:
rf = RandomForestClassifier(n_estimators = 100, max_depth = 2)
rf.fit(X_train_smote, y_train_smote)

# perm_rf = PermutationImportance(rf, random_state = 1).fit(X_test_attack, y_test_attack)
eli5.show_weights(rf, feature_names = X.columns.tolist())

Weight,Feature
0.6409 ± 0.8339,IAT
0.2947 ± 0.7571,ID
0.0553 ± 0.2412,Payload
0.0090 ± 0.0453,DLC
